In [3]:

from pathlib import Path
import sys
import pandas as pd
from Codes.sourcecode.generate_demand_metrics import compute_metrics

REPO_ROOT = Path('..').resolve()
DEMAND_PROFILES_ROOT = REPO_ROOT / 'Codes' / 'Output' / 'DemandProfiles'
SUMMARY_FILE = REPO_ROOT / 'Codes' / 'Data' / '1R1C1P1S_filtered_filtered.csv'
OUTPUT_FILE = REPO_ROOT / 'Codes' / 'Output' /'DemandProfiles'/ 'demand_metrics_summary.csv'

if str(REPO_ROOT) not in sys.path:
    sys.path.append(str(REPO_ROOT))

REPO_ROOT, DEMAND_PROFILES_ROOT, SUMMARY_FILE, OUTPUT_FILE

(WindowsPath('E:/GitHubProjects/LV network'),
 WindowsPath('E:/GitHubProjects/LV network/Codes/Output/DemandProfiles'),
 WindowsPath('E:/GitHubProjects/LV network/Codes/Data/1R1C1P1S_filtered_filtered.csv'),
 WindowsPath('E:/GitHubProjects/LV network/Codes/Output/DemandProfiles/demand_metrics_summary.csv'))

In [5]:


metrics = compute_metrics(
    demand_profiles_root=DEMAND_PROFILES_ROOT,
    summary_path=SUMMARY_FILE,
)
metrics.head()

,Dwelling ID,Device(HHP/mHP+capacity),Tariff Type,Weather(mild/extreme),Peak Electricity Consumption (Feb 11) 1600-1900,Total Electricity Consumption (Feb10-12),Total gas Consumption (Feb10-12),R(K/W),C(J/K),g(m^2)
0,EOH0003,HHP 4 kW,Time-of-use,mild,0.0,31.064,0.000,0.007548,4.091954e+07,13.439900
1,EOH0014,HHP 4 kW,Time-of-use,mild,0.0,49.187,4.439,0.006439,1.818650e+07,0.249881
2,EOH0018,HHP 4 kW,Time-of-use,mild,0.0,57.359,10.818,0.005212,2.267737e+07,2.288106
3,EOH0021,HHP 4 kW,Time-of-use,mild,0.0,65.622,38.559,0.004268,9.130042e+07,0.000001
4,EOH0025,HHP 4 kW,Time-of-use,mild,0.0,18.655,0.000,0.017066,3.422080e+07,0.000001


In [6]:

# Save to CSV if desired
metrics.to_csv(OUTPUT_FILE, index=False)
print(f'Saved {len(metrics)} rows to {OUTPUT_FILE}')

Saved 4064 rows to E:\GitHubProjects\LV network\Codes\Output\DemandProfiles\demand_metrics_summary.csv


In [3]:
metrics

,Dwelling ID,Device(HHP/mHP+capacity),Tariff Type,Weather(mild/extreme),Peak Electricity Consumption (Feb 11) 1600-1900,Total Electricity Consumption (Feb10-12),Total gas Consumption (Feb10-12),R(K/W),C(J/K),g(m^2)
0,EOH0003,HHP 4 kW,Time-of-use,mild,0.000,31.064,0.000,0.007548,4.091954e+07,13.439900
1,EOH0014,HHP 4 kW,Time-of-use,mild,0.000,49.187,4.439,0.006439,1.818650e+07,0.249881
2,EOH0018,HHP 4 kW,Time-of-use,mild,0.000,57.359,10.818,0.005212,2.267737e+07,2.288106
3,EOH0021,HHP 4 kW,Time-of-use,mild,0.000,65.622,38.559,0.004268,9.130042e+07,0.000001
4,EOH0025,HHP 4 kW,Time-of-use,mild,0.000,18.655,0.000,0.017066,3.422080e+07,0.000001
...,...,...,...,...,...,...,...,...,...,...
4059,EOH3183,mHP 7 kW,Flat,extreme,2.905,30.583,0.000,0.012392,2.410693e+07,2.869670
4060,EOH3186,mHP 8.5 kW,Flat,extreme,6.683,151.297,0.000,0.002672,9.716478e+07,1.365901
4061,EOH3188,mHP 7 kW,Flat,extreme,4.089,69.910,0.000,0.005384,1.884267e+07,5.504479
4062,EOH3196,mHP 7 kW,Flat,extreme,4.241,75.305,0.000,0.005005,2.778700e+07,7.022169


In [5]:
from Codes.sourcecode.analyze_peak_reduction import (
    HP_PROPORTION_TOU_COL,
    compute_peak_reduction,
    plot_histograms,
    plot_scatter_matrix,
)

metrics = pd.read_csv(OUTPUT_FILE)

peak_reduction = compute_peak_reduction(metrics)
peak_reduction.to_csv(
    DEMAND_PROFILES_ROOT / 'peak_demand_reduction_extreme.csv', index=False
)

hist_paths = plot_histograms(
    peak_reduction,
    DEMAND_PROFILES_ROOT / 'figures',
    metric_label="Peak time demand response(kWh)",
)

scatter_path = plot_scatter_matrix(
    peak_reduction,
    DEMAND_PROFILES_ROOT / 'figures' / 'peak_demand_reduction_scatter.png',
    metric_label="Peak time demand response(kWh)",
)

hhp_only = peak_reduction[
    peak_reduction['Technology'].astype(str).str.upper() == 'HHP'
]

if not hhp_only.empty:
    hp_hist_paths = plot_histograms(
        hhp_only,
        DEMAND_PROFILES_ROOT / 'figures',
        metric_col=HP_PROPORTION_TOU_COL,
        metric_label='Heat pump heat proportion (Time-of-use)',
        file_prefix='hp_heat_proportion_hist',
        units=None,
    )
    hp_scatter_path = plot_scatter_matrix(
        hhp_only,
        DEMAND_PROFILES_ROOT / 'figures' / 'hp_heat_proportion_scatter.png',
        metric_col=HP_PROPORTION_TOU_COL,
        metric_label='Heat pump heat proportion (Time-of-use)',
    )
else:
    print('No HHP rows available for heat proportion plots; skipping figure generation.')
    hp_hist_paths = {}
    hp_scatter_path = None

print('Saved peak reduction histogram(s):')
for tech, path in hist_paths.items():
    print(f' - {tech}: {path}')
print(f'Peak reduction scatter matrix: {scatter_path}')

if hp_hist_paths:
    print('Saved heat proportion histogram(s):')
    for tech, path in hp_hist_paths.items():
        print(f' - {tech}: {path}')
else:
    print('No heat proportion histograms generated.')
if hp_scatter_path:
    print(f'Heat proportion scatter matrix: {hp_scatter_path}')
else:
    print('Heat proportion scatter matrix not generated.')

peak_reduction.head()


E:\GitHubProjects\LV network\Codes\sourcecode\analyze_peak_reduction.py:220: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):


Saved peak reduction histogram(s):
 - HHP: E:\GitHubProjects\LV network\Codes\Output\DemandProfiles\figures\peak_demand_reduction_hist_hhp.png
 - mHP: E:\GitHubProjects\LV network\Codes\Output\DemandProfiles\figures\peak_demand_reduction_hist_mhp.png
Peak reduction scatter matrix: E:\GitHubProjects\LV network\Codes\Output\DemandProfiles\figures\peak_demand_reduction_scatter.png
Saved heat proportion histogram(s):
 - HHP: E:\GitHubProjects\LV network\Codes\Output\DemandProfiles\figures\hp_heat_proportion_hist_hhp.png
Heat proportion scatter matrix: E:\GitHubProjects\LV network\Codes\Output\DemandProfiles\figures\hp_heat_proportion_scatter.png


,Dwelling ID,Technology,Capacity kW,Flat,Time-of-use,HP Heat Proportion (Flat),HP Heat Proportion (Time-of-use),R(K/W),C(J/K),g(m^2),HTC (1/R),Thermal Constant (R*C),Peak Demand Reduction (Flat -> Time-of-use) [kWh]
0,EOH0003,HHP,4.0,1.994,0.0,1.000000,1.000000,0.007548,4.091954e+07,13.439900,132.482394,308867.737608,1.994
1,EOH0014,HHP,4.0,2.910,0.0,0.990413,0.925865,0.006439,1.818650e+07,0.249881,155.292951,117110.930034,2.910
2,EOH0018,HHP,4.0,3.317,0.0,0.935754,0.847098,0.005212,2.267737e+07,2.288106,191.866893,118193.215518,3.317
3,EOH0021,HHP,4.0,3.429,0.0,0.820875,0.705056,0.004268,9.130042e+07,0.000001,234.285017,389698.093829,3.429
4,EOH0025,HHP,4.0,1.810,0.0,1.000000,1.000000,0.017066,3.422080e+07,0.000001,58.596817,584004.364209,1.810
